In [2]:
# %%
import geopandas as gpd
import numpy as np

shp="/vol/home/s2465922/data/community_areas/geo_export_2d6ca2cd-726e-4f78-9dc4-b833b8f52d77.shp"

community=gpd.read_file(shp)

community['centroid']=community['geometry'].centroid

# %%
# kw=ps.lib.weights.Kernel.from_dataframe(community, bandwidth=0.09, 
#     function='gaussian',ids='area_num_1')

# %%
class NeighborEdge:

  def euclidean_dist(self,p1,p2):
    p1=np.array(p1)
    p2=np.array(p2)
    d=np.sqrt(sum((p1-p2)**2))
    return d

  def hour_diff(self,t1,t2):
    diff=abs(t1-t2)
    return min(diff,24-diff)

  def kernel(self, d, bw):
    d_=d/bw
    kw=0 if d_>1 else np.exp((-d_*d_/2))
    return kw

  def create_edges(self,node_dict,bw,type):
    edges=[]
    nodes=list(node_dict.keys())

    for i in range(len(nodes)-1):
      for j in range(i+1, len(nodes)):
        v1, v2=node_dict[nodes[i]], node_dict[nodes[j]]
        if type=='r':
          kw=self.kernel(self.euclidean_dist(v1,v2), bw)
        if type=='h':
          kw=self.kernel(self.hour_diff(v1,v2), bw)
        if kw>0:
          edges.append((nodes[i], nodes[j], kw))
          edges.append((nodes[j], nodes[i], kw))

    return edges

  def save_edges(self, edge_type, edge_lst):
    file=edge_type+'_edges.txt'
    with open(file, 'w') as f:
      for edge in  edge_lst:
          f.write("\t".join(str(e) for e in edge))
          f.write("\n")

community['point']=community['centroid'].apply(lambda p: (p.x, p.y))
points=community.set_index('area_num_1')['point'].to_dict()

hours=dict({(i,h) for i,h in enumerate(np.arange(24))})

ne=NeighborEdge()

hh_edges=ne.create_edges(hours,bw=4, type='h')

rr_edges=ne.create_edges(points,bw=0.05, type='r')

ne.save_edges('hh', hh_edges)

ne.save_edges('rr', rr_edges)



# %%


# %%


In [31]:
len(rr_edges)

558

In [32]:
def hour_diff(t1,t2):
  diff=abs(t1-t2)
  return min(diff,24-diff)

In [35]:
hour_diff(0,22)

2